In [14]:
import math
import pandas as pd

with open('dataset/sw.txt') as file:
    stop_words = []
    for item in file:
        stop_words.append(item.rstrip('\n'))

titles = []
texts = []
unigram = []
bigram = []
epsilon = 0.000000000001
epsilon2 = 0.00000001
classes_dict = dict()
classes = []
classes_words_num_dict = dict()
classes_phrases_num_dict = dict()
Lambda1 = 0.8
Lambda2 = 0.2
unknown_word_hits = 0
unknown_phrase_hits = 0

word_each_class_num = dict()
phrase_each_class_num = dict()

In [15]:
pure_data = pd.read_csv('dataset/reviews_train.csv')
texts = pure_data["Review"].dropna().values
titles = pure_data["Label"].dropna().values

for title in titles:
    classes_dict[title] = classes_dict.get(title, 0) + 1

In [16]:
train_doc_size = len(titles)
classes = classes_dict.keys()
print(classes_dict)
print(classes)

{'pos': 102, 'neg': 77}
dict_keys(['pos', 'neg'])


In [17]:
for j in range(len(titles)):
    text = texts[j]
    title = titles[j]
    words = text.split(" ")
    counter = dict()
    counter2 = dict()

    ii = 0
    word2 = ""
    for word in words:
        word = word.lower()
        if word in stop_words or word2 in stop_words:
            continue
        ii += 1
        counter[word] = counter.get(word, 0.0) + 1.0
        phrase = word2 + word
        counter2[phrase] = counter2.get(phrase, 0.0) + 1.0
        if word_each_class_num.get(word, 0) == 0:
            word_each_class_num[word] = dict()

        word_each_class_num[word][title] = word_each_class_num[word].get(title, 0) + 1

        if phrase_each_class_num.get(phrase, 0) == 0:
            phrase_each_class_num[phrase] = dict()

        phrase_each_class_num[phrase][title] = phrase_each_class_num[phrase].get(title, 0) + 1

        word2 = word

    counter['words_number'] = ii
    counter2['phrases_number'] = ii
    unigram.append(counter)
    bigram.append(counter2)

all_words_num = 0
all_phrases_num = 0
for j in range(train_doc_size):
    all_words_num += unigram[j]['words_number']
    all_phrases_num += bigram[j]['phrases_number']

In [18]:
test_titles = []
test_texts = []
test_answers = []


pure_data_test = pd.read_csv('dataset/reviews_test.csv')
test_texts = pure_data_test["Review"].dropna().values
test_titles = pure_data_test["Label"].dropna().values


In [19]:
def word_prob(wordd):
    word_repetition = 0.0
    for i in range(train_doc_size):
        word_repetition += unigram[i].get(wordd, epsilon2)
    return word_repetition / all_words_num


def phrase_prob(phrasee):
    phrase_repetition = 0.0
    for i in range(train_doc_size):
        phrase_repetition += bigram[i].get(phrasee, epsilon2)
    return phrase_repetition / all_phrases_num


def unigram_class_words_num(_class):
    class_words_num = 0.0
    for ii in range(train_doc_size):
        if titles[ii] == _class:
            class_words_num += unigram[ii]['words_number']
    return class_words_num


def bigram_class_phrases_num(_class):
    class_phrases_num = 0.0
    for ii in range(train_doc_size):
        if titles[ii] == _class:
            class_phrases_num += bigram[ii]['phrases_number']
    return class_phrases_num


for _class_ in classes:
    classes_words_num_dict[_class_] = unigram_class_words_num(_class_)
    classes_phrases_num_dict[_class_] = bigram_class_phrases_num(_class_)


def prob_word_class(word, classs):
    global unknown_word_hits
    if word_each_class_num.get(word, 0) == 0:
        unknown_word_hits += 1
        return epsilon
    if word_each_class_num[word].get(classs, 0) == 0:
        return epsilon
    else:
        return word_each_class_num[word][classs]/classes_words_num_dict[classs]


def prob_phrase_class(word1, word2, classs):
    global unknown_phrase_hits
    phrase = word2+word1
    if phrase_each_class_num.get(phrase, -1) == -1:
        unknown_phrase_hits += 1
        return epsilon2
    if phrase_each_class_num[phrase].get(classs, -1) == -1:
        return epsilon2
    else:
        if word_each_class_num.get(word2, -1) == -1:
            unknown_phrase_hits += 1
            return epsilon2
        if word_each_class_num[word2].get(classs, -1) == -1:
            return epsilon2
        else:
            return phrase_each_class_num[phrase][classs]/word_each_class_num[word2][classs]


In [20]:
for test_text in test_texts:
    test_text_words = test_text.split(" ")
    best_prob = 0.0
    class_answer = ""
    curr = 0
    for _classs in classes:
        unigram_class_prob = classes_words_num_dict[_classs]/all_words_num
        bigram_class_prob = classes_phrases_num_dict[_classs] / all_phrases_num
        # unigram_class_prob = classes_dict[_classs] / train_doc_size
        # bigram_class_prob = classes_dict[_classs] / train_doc_size
        unigram_sigma = 0.0
        bigram_sigma = 0.0
        wrd2 = ""
        for wrd in test_text_words:
            wrd = wrd.lower()
            if wrd in stop_words or wrd2 in stop_words:
                continue
            unigram_sigma += math.log10(prob_word_class(wrd, _classs))
            phrase = wrd2+wrd
            bigram_sigma += math.log10(prob_phrase_class(wrd, wrd2, _classs))
            wrd2 = wrd
        unigram_prob = math.log10(unigram_class_prob) + unigram_sigma
        bigram_prob = math.log10(bigram_class_prob) + bigram_sigma
        prob = Lambda1 * unigram_prob + Lambda2 * bigram_prob

        if curr == 0:
            best_prob = prob
            class_answer = _classs
        elif prob > best_prob:
            best_prob = prob
            class_answer = _classs
        curr += 1

    test_answers.append(class_answer)

In [21]:
TP = dict()
FP = dict()
true_answers = 0
for i in range(len(test_titles)):
    if test_titles[i] == test_answers[i]:
        true_answers += 1
        TP[test_titles[i]] = TP.get(test_titles[i], 0) + 1
    else:
        if FP.get(test_titles[i], 0) == 0:
            FP[test_titles[i]] = dict()
        FP[test_titles[i]][test_answers[i]] = FP[test_titles[i]].get(test_answers[i], 0) + 1

In [22]:
print("test_doc_size: ", len(test_titles))
print("unknown word hits is: ", unknown_word_hits)
print("unknown phrase hits is: ", unknown_phrase_hits)
print("Total Accuracy is : ", true_answers/len(test_titles)*100)
recall = dict()
for class_ in classes:
    TPP = TP[class_]
    keys = FP[class_].keys()
    FPP = 0
    for key in keys:
        FPP += FP[class_][key]
    recall[class_] = TPP/(TPP+FPP)

precision = dict()
for class_ in classes:
    TPP = TP[class_]
    FPP = 0
    for cls in classes:
        if cls != class_:
            FPP += FP[cls].get(class_, 0)
    precision[class_] = TPP/(TPP+FPP)

print('Recall         Precision     F-measure')
for class_ in classes:
    rcal = int(recall[class_]*10000)
    prcsion = int(precision[class_]*10000)
    rcal = rcal/10000
    prcsion = prcsion/10000
    fmeasure = int(2 * rcal * prcsion / (rcal + prcsion) * 10000)
    fmeasure = fmeasure/10000
    space = " " * (15-len(class_))
    print(rcal, "       ", prcsion, "        ", fmeasure, space, class_)



test_doc_size:  20
unknown word hits is:  902
unknown phrase hits is:  1752
Total Accuracy is :  60.0
Recall         Precision     F-measure
0.7777         0.5384          0.6362              pos
0.4545         0.7142          0.5554              neg
